In [4]:
from __future__ import print_function
import os
import cv2
from google.cloud import vision
import io
from google.oauth2 import service_account
import us
import webcolors
import binascii
import struct
import numpy as np
import scipy
import scipy.misc
import scipy.cluster
import imageio
# Importing the modules for collecting and building the dataset
"""import time
import os
import re

# Web-scraping...
from urllib.request import Request, urlopen
from urllib.request import URLError, HTTPError
"""
import shutil
# Image processing...
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from autocorrect import Speller
from keras.models import load_model

In [5]:
from PIL import Image

In [6]:
def PolygonArea(corners):
    n = len(corners) # of corners
    area = 0.0
    for i in range(n):
        j = (i + 1) % n
        area += corners[i][0] * corners[j][1]
        area -= corners[j][0] * corners[i][1]
    area = abs(area) / 2.0
    return area

In [28]:
def googleAPI(path):
    key = '/Users/kunal/Documents/AAPersonalAIPROJECT/Cardetection/GOOGLEAPI/key.json'
    credentials = service_account.Credentials.from_service_account_file(key)
    client = vision.ImageAnnotatorClient(credentials=credentials)
    # [START vision_python_migration_text_detection]
    with io.open(path, 'rb') as image_file:
        content = image_file.read()
    image = vision.types.Image(content=content)
    response = client.text_detection(image=image)
    texts = response.text_annotations
    img = cv2.imread(path)
    w, h, dis = img.shape
    areaTotal = w*h
    result = []
    count = 1
    for text in texts:
        verticeArray=[]
        for vertex in text.bounding_poly.vertices:
            verticeSingle = (vertex.x, vertex.y)
            verticeArray.append(verticeSingle) 
        area = PolygonArea(verticeArray)
        percent = ((area/areaTotal)*100)
        totalText = [text.description, verticeArray, area, percent]

        result.append(totalText)
        count+=1
        #print("\n")
    if response.error.message:
        print(response.error.message)
        print("NOTHINGDETECTED")
    return result

In [29]:
def cleaningGoogleAPI(result):
    stateDetectionArray = []
    for i in result:
        stateDetectionArray.append(i[0])
    spell = Speller(lang='en')
    countyList = ["Fulton"]
    ct=0
    removeARRAY = []
    for i in stateDetectionArray:
        for j in us.states.STATES:
            if spell(i.lower()) == str(j).lower():
                removeARRAY.append(i)
        for k in countyList:
            if spell(i.lower()) == k.lower():
                removeARRAY.append(i)
        if len(i) > 9:
            removeARRAY.append(i)
        ct+=1
    removeARRAY = list(set(removeARRAY))
    resultClean = result
    num = 0
    for i in result: 
        for j in removeARRAY:
            if i[0] == j:
                resultClean.pop(num)
        num+=1
    perArrayRemove = []
    for i in resultClean:
        if i[3] < 1:
            perArrayRemove.append(i)
    ctFR = 0
    for i in result:
        for j in perArrayRemove:
            if i == j:
                resultClean.pop(ctFR)
        ctFR+=1
    LISENCEPLATE =""
    if len(resultClean) == 1:
        LISENCEPLATE = resultClean[0][0]
    elif len(resultClean) > 1:
        print("CHECK")
        LISENCEPLATE = "Check"
    else:
        LISENCEPLATE = "N/A"
    return LISENCEPLATE, resultClean

In [30]:
def colorDetect(path):
    NUM_CLUSTERS = 5
    #print('Reading image')
    im = Image.open(path)
    im = im.resize((150, 150))      # optional, to reduce time
    ar = np.asarray(im)
    shape = ar.shape
    ar = ar.reshape(scipy.product(shape[:2]), shape[2]).astype(float)
    #print('Finding clusters')
    codes, dist = scipy.cluster.vq.kmeans(ar, NUM_CLUSTERS)
    #print('Cluster centres:\n', codes)

    vecs, dist = scipy.cluster.vq.vq(ar, codes)         # assign codes
    counts, bins = scipy.histogram(vecs, len(codes))    # count occurrences

    index_max = scipy.argmax(counts)                    # find most frequent
    peak = codes[index_max]
    colour = binascii.hexlify(bytearray(int(c) for c in peak)).decode('ascii')
    #print('Most frequent is %s (#%s)' % (peak, colour))
    return codes, peak, colour

In [31]:
def closest_colour(requested_colour):
    min_colours = {}
    for key, name in webcolors.css3_hex_to_names.items():
        r_c, g_c, b_c = webcolors.hex_to_rgb(key)
        rd = (r_c - requested_colour[0]) ** 2
        gd = (g_c - requested_colour[1]) ** 2
        bd = (b_c - requested_colour[2]) ** 2
        min_colours[(rd + gd + bd)] = name
    return min_colours[min(min_colours.keys())]

def get_colour_name(requested_colour):
    try:
        closest_name = actual_name = webcolors.rgb_to_name(requested_colour)
    except ValueError:
        closest_name = closest_colour(requested_colour)
        actual_name = None
    return actual_name, closest_name
def getFINALCOLOR(peak):
    r = int(peak[0])
    g = int(peak[1])
    b = int(peak[2])
    requested_colour = (r, g, b)
    actual_name, closest_name = get_colour_name(requested_colour)

    print ("Actual colour name:", actual_name, ", closest colour name:", closest_name)
    COLORNAME1 = actual_name
    COLORNAME2 = closest_name
    if COLORNAME1 != "":
        COLORNAME1 = "N/A"
    return COLORNAME1, COLORNAME2 


In [32]:
# I will need that later to re-convert my flattened images to their previous state
def ImageConvert(n, i):
    img_x = img_y = 50
    im_ex = i.reshape(n, img_x, img_y, 3)
    im_ex = im_ex.astype('float32') / 255
    im_ex = np.subtract(im_ex, 0.5)
    im_ex = np.multiply(im_ex, 2.0)
    return im_ex

In [33]:
def symbol(path):
    cars = ['Alfa Romeo', 'Audi', 'BMW', 'Chevrolet', 'Citroen', 'Dacia', 'Daewoo', 'Dodge',
            'Ferrari', 'Fiat', 'Ford', 'Honda', 'Hyundai', 'Jaguar', 'Jeep', 'Kia', 'Lada',
            'Lancia', 'Land Rover', 'Lexus', 'Maserati', 'Mazda', 'Mercedes', 'Mitsubishi',
            'Nissan', 'Opel', 'Peugeot', 'Porsche', 'Renault', 'Rover', 'Saab', 'Seat',
            'Skoda', 'Subaru', 'Suzuki', 'Tata', 'Tesla', 'Toyota', 'Volkswagen', 'Volvo']
    model = load_model('/Users/kunal/Documents/Cardetection/Jupyternotebook/symbolMODEL.h5')
    img_x = img_y = 50
    im = Image.open(path).convert("RGB")
    new_im = np.array(im.resize((50,50))).flatten()
    m = int(model.predict_classes(ImageConvert(1, new_im), verbose=0))
    LOGODETECTED = cars[m]
    return LOGODETECTED

In [34]:
def runALL(path):
    result = googleAPI(path)
    LISENCEPLATE, resultClean = cleaningGoogleAPI(result)
    codes, peak, colour = colorDetect(path)
    COLORNAME1, COLORNAME2 = getFINALCOLOR(peak)
    LOGODETECTED = symbol(path)
    return (result, LISENCEPLATE, resultClean, codes, peak, colour, COLORNAME1, COLORNAME2, LOGODETECTED)


In [35]:
def extract_image_one_fps(video_source_path):
    vidcap = cv2.VideoCapture(video_source_path)
    count = 0
    success = True
    while success:
        vidcap.set(cv2.CAP_PROP_POS_MSEC,(count*4000)) # 2 second***   
        success,image = vidcap.read()
        ## Stop when last frame is identified
        image_last = cv2.imread("frame{}.png".format(count-1))
        if np.array_equal(image, image_last):
            break
        cv2.imwrite("frame%d.png" % count, image)     # save frame as PNG file
        print( '{}.sec reading a new frame:{}'.format(count,success))
        count += 1

In [36]:
def appendthefiles(a,b,c,d,e,f,g,h,i,j):
    LisencePLATEARRAYARRAY.append(a)
    LISENCEPLATETEXTARRAY.append(b)
    LisencePLATECLEANARRAY.append(c)
    MAINCOLORSARRAY.append(d)
    MAINCOLORRGBARRAY.append(e)
    MAINCOLORHEXARRAY.append(f)
    COLORNAME1ARRAY.append(g)
    COLORNAME2ARRAY.append(h)
    LOGOARRAY.append(i)
    NAMEOFFILEARRAY.append(j)

In [37]:
LisencePLATEARRAYARRAY,LISENCEPLATETEXTARRAY,LisencePLATECLEANARRAY,MAINCOLORSARRAY,MAINCOLORRGBARRAY, MAINCOLORHEXARRAY, COLORNAME1ARRAY, COLORNAME2ARRAY, LOGOARRAY, NAMEOFFILEARRAY = ([] for i in range(10))

In [38]:
dirTOTAL = ('/Users/kunal/Documents/AAPersonalAIPROJECT/Cardetection/RunningFINAL/testvid/')
os.chdir(dirTOTAL)

In [39]:
path = '/Users/kunal/Documents/AAPersonalAIPROJECT/Cardetection/RunningFINAL/testvid/frame1.png'

In [40]:
runALL(path)

AttributeError: module 'google.cloud.vision' has no attribute 'types'

In [24]:
%%time
numRUNS = 0
for i in os.listdir(dirTOTAL):
    if i.endswith(".mp4"):
        j = i
        pathName = dirTOTAL + i.replace('.mp4', '')
        print(i)
        print(pathName)
        os.mkdir(pathName)
        shutil.move(i, pathName)
        finalDir = pathName + "/" + i
        os.chdir(pathName)
        extract_image_one_fps(finalDir)
        countName = 0
        for i in os.listdir("."):
            if i.endswith(".png"):
                if os.path.getsize(i) != 0:
                    img = Image.open(i)
                    w1, h1 = img.size 
                    img = img.resize((int(w1/2),int(h1/2)),Image.ANTIALIAS)
                    fileName = "frameReduced" + str(countName) + ".jpg"
                    img.save(fileName,optimize=True,quality=95)
                    #os.remove(fileName)
                    a,b,c,d,e,f,g,h,i = runALL(fileName)
                    appendthefiles(a,b,c,d,e,f,g,h,i,j)
                    numRUNS += 1
            countName+=1
        os.chdir(dirTOTAL)
        print(finalDir)
    #(result, LISENCEPLATE, resultClean, codes, peak, colour, COLORNAME1, COLORNAME2, LOGODETECTED)
print(numRUNS)

testvid.mp4
/Users/kunal/Documents/AAPersonalAIPROJECT/Cardetection/RunningFINAL/testvid/testvid
0.sec reading a new frame:True
1.sec reading a new frame:True
2.sec reading a new frame:True
3.sec reading a new frame:True
4.sec reading a new frame:True


error: OpenCV(4.5.1) C:\Users\appveyor\AppData\Local\Temp\1\pip-req-build-kh7iq4w7\opencv\modules\imgcodecs\src\loadsave.cpp:753: error: (-215:Assertion failed) !_img.empty() in function 'cv::imwrite'


In [17]:
LisencePLATEARRAYARRAY,LISENCEPLATETEXTARRAY,LisencePLATECLEANARRAY,MAINCOLORSARRAY,MAINCOLORRGBARRAY, MAINCOLORHEXARRAY, COLORNAME1ARRAY, COLORNAME2ARRAY, LOGOARRAY, NAMEOFFILEARRAY
data = {"Video File Name": NAMEOFFILEARRAY,
        "Lisence Plate": LISENCEPLATETEXTARRAY,
        "Color " : COLORNAME2ARRAY,
        "LOGO" : LOGOARRAY,
        "Color 2" : COLORNAME1ARRAY,
        "Hex Color" : MAINCOLORHEXARRAY,
        "Color Array": MAINCOLORRGBARRAY,
        "Total Colors" : MAINCOLORSARRAY,
        "Lisence Plate Total Edited" : LisencePLATECLEANARRAY,
        "Lisence Plate Total" : LisencePLATEARRAYARRAY
       }

In [19]:
df = pd.DataFrame(data) 
df.to_excel("FINALOUTPUT.xlsx")  
df

,Video File Name,Lisence Plate,Color,LOGO,Color 2,Hex Color,Color Array,Total Colors,Lisence Plate Total Edited,Lisence Plate Total
0,testvid.mp4,N/A,gainsboro,Dodge,N/A,ddd8d5,"[221.45021892252046, 216.7445269369884, 213.07...","[[65.40450667250055, 59.57886676875957, 58.638...",[],[]
1,testvid.mp4,N/A,darkslategrey,Dodge,N/A,544440,"[84.13396760092569, 68.47544355875546, 64.1144...","[[84.13396760092569, 68.47544355875546, 64.114...",[],[]
2,testvid.mp4,Check,darkolivegreen,Dodge,N/A,534038,"[83.63045256270448, 64.48541439476554, 56.4133...","[[239.0940366972477, 231.9822247706422, 227.93...","[[ORCIA, [(926, 1079), (921, 958), (948, 957),...","[[ORCIA, [(926, 1079), (921, 958), (948, 957),..."
3,testvid.mp4,Check,dimgrey,Dodge,N/A,655c5a,"[101.7021721454292, 92.03380183528866, 90.6253...","[[63.89702389269247, 63.0371664105072, 66.5587...","[[Piair, [(1018, 351), (1014, 289), (1051, 287...","[[Piair, [(1018, 351), (1014, 289), (1051, 287..."
4,testvid.mp4,Check,dimgrey,Kia,N/A,786e68,"[120.65939803439804, 110.63313882063882, 104.5...","[[120.65939803439804, 110.63313882063882, 104....","[[organi, [(1291, 266), (1331, 273), (1329, 28...","[[organi, [(1291, 266), (1331, 273), (1329, 28..."
5,testvid_Trim.mp4,Check,darkslategrey,Dodge,N/A,503f3a,"[80.44715447154472, 63.37756648752928, 58.8867...","[[36.144020574367765, 32.44899271324475, 35.79...","[[State, [(1000, 914), (1043, 912), (1044, 932...","[[State, [(1000, 914), (1043, 912), (1044, 932..."
6,testvid_Trim.mp4,Check,darkolivegreen,Dodge,N/A,523c32,"[82.16037867186412, 60.30224140331338, 50.4566...","[[118.74733207784055, 101.78907721280602, 100....","[[DYNA\nMDX\n, [(227, 608), (447, 608), (447, ...","[[DYNA\nMDX\n, [(227, 608), (447, 608), (447, ..."
7,testvid_Trim.mp4,Check,darkslategrey,Renault,N/A,3c3a3e,"[60.32207293666027, 58.71170825335893, 62.1460...","[[227.52705410821645, 228.75350701402806, 232....","[[AM, [(446, 458), (637, 455), (640, 638), (44...","[[AM, [(446, 458), (637, 455), (640, 638), (44..."
8,VID_20200316_124813.mp4,Check,silver,Porsche,N/A,c8c7c4,"[200.46613545816734, 199.69588313413016, 196.3...","[[81.96615189337847, 76.19928072773429, 75.971...","[[organic, [(1187, 336), (1246, 357), (1237, 3...","[[organic, [(1187, 336), (1246, 357), (1237, 3..."
9,VID_20200316_124813.mp4,ORGIA,dimgrey,Dodge,N/A,636063,"[99.74551299223145, 96.30016072863648, 99.0478...","[[76.1259958689879, 72.40070817350251, 72.6714...","[[ORGIA, [(855, 1075), (871, 760), (961, 765),...","[[ORGIA, [(855, 1075), (871, 760), (961, 765),..."
